In [24]:
import pulp

In [25]:
n = 7
soc = [20,20,20,20,20,20,20]
stations = [20,30,50]
speed = 10
charging_rate = 10
arrival_time = [0,2,4,6,8,10,12]
T = 1000

In [26]:
#initiate the model
cs = pulp.LpProblem("station",pulp.LpMinimize)
vehicles=[]
#add variables
soc_vehicle = {}
arrival_time_vehicle = {}
for i in range(n):
    vehicles.append(f"vehicle{i}")
    soc_vehicle[f"vehicle{i}"] = soc[i]
    arrival_time_vehicle[f"vehicle{i}"] = arrival_time[i]
M = 100000
print(vehicles)
#arrival time at stations
a1 = pulp.LpVariable.dict("Arrival Time 1",[vehicle for vehicle in vehicles],lowBound=0,upBound=T,cat=pulp.LpInteger)
a2 = pulp.LpVariable.dict("Arrival Time 2",[vehicle for vehicle in vehicles],lowBound=0,upBound=T,cat=pulp.LpInteger)
# servicing time at stations
s1 = pulp.LpVariable.dict("Service Time 1",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
s2 = pulp.LpVariable.dict("Service Time 2",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
#initial charging time at sations
u1 = pulp.LpVariable.dict("initial charge time 1",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
u2 = pulp.LpVariable.dict("initial charge time 2",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
#final charge time at stations
d1 = pulp.LpVariable.dict("final charge time 1",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
d2 = pulp.LpVariable.dict("final charge time 2",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
# sigma- temporal ordering of vehicles
o1 = pulp.LpVariable.dict("sigma 1",[vehicle1+vehicle2 for vehicle1 in vehicles for vehicle2 in vehicles if vehicle1 != vehicle2],lowBound=0,cat=pulp.LpBinary)
o2 = pulp.LpVariable.dict("sigma 2",[vehicle1+vehicle2 for vehicle1 in vehicles for vehicle2 in vehicles if vehicle1 != vehicle2],lowBound=0,cat=pulp.LpBinary)
# to see if vehicle is gonna use the respective station
sb1 = pulp.LpVariable.dict("Service Time 1 binary",[vehicle for vehicle in vehicles],cat=pulp.LpBinary)
sb2 = pulp.LpVariable.dict("Service Time 2 binary",[vehicle for vehicle in vehicles],cat=pulp.LpBinary)




['vehicle0', 'vehicle1', 'vehicle2', 'vehicle3', 'vehicle4', 'vehicle5', 'vehicle6']


In [27]:
#objective function
cs += pulp.lpSum([d1[vehicle]-a1[vehicle] + d2[vehicle]-a2[vehicle]] for vehicle in vehicles)
#blending+= (pulp.lpSum([4.32*ing_weight[(i,"pork")]+2.46*ing_weight[(i,"wheat")]+1.86*ing_weight[(i,"starch")] for i in sausages]))

In [28]:
#constraints
for vehicle in vehicles:
    #departure = service + charge start
    cs += d1[vehicle] == s1[vehicle]+u1[vehicle]
    cs += d2[vehicle] == s2[vehicle]+u2[vehicle]
    
    # charge start > arival time
    cs += u1[vehicle]>=a1[vehicle]
    cs += u2[vehicle]>=a2[vehicle]
    cs += u2[vehicle]<=T-s2[vehicle]
    
    # arrival times of each station
    cs += a1[vehicle] == arrival_time_vehicle[vehicle]+(stations[0]/speed)
    cs += a2[vehicle] == (stations[1]/speed) + d1[vehicle]
    
    # ensuring soc capabilities
    cs += charging_rate*s1[vehicle] + soc_vehicle[vehicle]>=stations[0]+stations[1]
    cs += charging_rate*s1[vehicle] + soc_vehicle[vehicle]<=100
    cs += charging_rate*(s2[vehicle]+s1[vehicle])+soc_vehicle[vehicle]>=stations[0]+stations[1]+stations[2]
    cs += charging_rate*(s2[vehicle]+s1[vehicle])+soc_vehicle[vehicle]-stations[0]-stations[1]<=100
    cs += M*sb1[vehicle] >= s1[vehicle]
    cs += M*sb1[vehicle] >=s1[vehicle]
    cs += sb1[vehicle] <= s1[vehicle]
    cs += M*sb2[vehicle] >=s2[vehicle]
    cs += sb2[vehicle] <= s2[vehicle]
for vehicle1 in vehicles:
    
    for vehicle2 in vehicles:
        if vehicle1!=vehicle2:
            #making sure o1[vehicle1+vehicle2] = 1 when vehicle1 in front of vehicle2
            cs += M*o1[vehicle1+vehicle2] >= a1[vehicle2]-a1[vehicle1]
            cs += M*(1-o1[vehicle1+vehicle2])>= a1[vehicle1]-a1[vehicle2]
            cs += M*o2[vehicle1+vehicle2] >= a2[vehicle2]-a2[vehicle1]
            cs += M*(1-o2[vehicle1+vehicle2]) >= a2[vehicle1]-a2[vehicle2]
            #implementing wait time
            cs += u1[vehicle2]-u1[vehicle1]-s1[vehicle1]>=(o1[vehicle1+vehicle2]-1)*T-M*(1-sb1[vehicle2])
            cs += u2[vehicle2]-u2[vehicle1]-s2[vehicle1]>=(o2[vehicle1+vehicle2]-1)*T-M*(1-sb2[vehicle2])
            #making sure only one vehicle is front of another
            cs += o1[vehicle1+vehicle2] + o1[vehicle2+vehicle1] >= 1
            cs += o2[vehicle1+vehicle2] + o2[vehicle2+vehicle1] >= 1




In [29]:
cs

station:
MINIMIZE
-1*Arrival_Time_1_vehicle0 + -1*Arrival_Time_1_vehicle1 + -1*Arrival_Time_1_vehicle2 + -1*Arrival_Time_1_vehicle3 + -1*Arrival_Time_1_vehicle4 + -1*Arrival_Time_1_vehicle5 + -1*Arrival_Time_1_vehicle6 + -1*Arrival_Time_2_vehicle0 + -1*Arrival_Time_2_vehicle1 + -1*Arrival_Time_2_vehicle2 + -1*Arrival_Time_2_vehicle3 + -1*Arrival_Time_2_vehicle4 + -1*Arrival_Time_2_vehicle5 + -1*Arrival_Time_2_vehicle6 + 1*final_charge_time_1_vehicle0 + 1*final_charge_time_1_vehicle1 + 1*final_charge_time_1_vehicle2 + 1*final_charge_time_1_vehicle3 + 1*final_charge_time_1_vehicle4 + 1*final_charge_time_1_vehicle5 + 1*final_charge_time_1_vehicle6 + 1*final_charge_time_2_vehicle0 + 1*final_charge_time_2_vehicle1 + 1*final_charge_time_2_vehicle2 + 1*final_charge_time_2_vehicle3 + 1*final_charge_time_2_vehicle4 + 1*final_charge_time_2_vehicle5 + 1*final_charge_time_2_vehicle6 + 0
SUBJECT TO
_C1: - Service_Time_1_vehicle0 + final_charge_time_1_vehicle0
 - initial_charge_time_1_vehicle0 = 0



In [30]:
cs.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/sushi/Desktop/internship/Routing/.venv/lib/python3.8/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/5022708850644068b23d2e3e339cd718-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/5022708850644068b23d2e3e339cd718-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 453 COLUMNS
At line 2099 RHS
At line 2548 BOUNDS
At line 2703 ENDATA
Problem MODEL has 448 rows, 154 columns and 1309 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 56 - 0.00 seconds
Cgl0003I 0 fixed, 48 tightened bounds, 49 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 19 tightened bounds, 35 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 3 tightened bounds, 17 strengthened rows, 0 substitutions
Cgl0004I processed model has 210 rows, 84 columns (84 integer (49 of which binary)) and 658 

1

In [31]:
pulp.LpStatus[cs.status]

'Optimal'

In [32]:
for vehicle in vehicles:
    print(f"s1: {s1[vehicle].varValue}, s2: {s2[vehicle].varValue}, a1,a2 : {a1[vehicle].varValue},{a2[vehicle].varValue}, u1,u2 :{u1[vehicle].varValue}.{u2[vehicle].varValue} d1,d2 :{d1[vehicle].varValue}.{d2[vehicle].varValue}")

s1: 3.0, s2: 5.0, a1,a2 : 2.0,8.0, u1,u2 :2.0.8.0 d1,d2 :5.0.13.0
s1: 4.0, s2: 4.0, a1,a2 : 4.0,12.0, u1,u2 :5.0.13.0 d1,d2 :9.0.17.0
s1: 5.0, s2: 3.0, a1,a2 : 6.0,17.0, u1,u2 :9.0.17.0 d1,d2 :14.0.20.0
s1: 3.0, s2: 5.0, a1,a2 : 8.0,20.0, u1,u2 :14.0.20.0 d1,d2 :17.0.25.0
s1: 5.0, s2: 3.0, a1,a2 : 10.0,25.0, u1,u2 :17.0.25.0 d1,d2 :22.0.28.0
s1: 3.0, s2: 5.0, a1,a2 : 12.0,28.0, u1,u2 :22.0.28.0 d1,d2 :25.0.33.0
s1: 8.0, s2: 0.0, a1,a2 : 14.0,36.0, u1,u2 :25.0.36.0 d1,d2 :33.0.36.0


In [12]:
print(pulp.value(cs.objective))

16.0
